In [1]:
import pandas as pd
import json
import os

data = pd.read_csv('data/results.csv')

In [3]:
data['date'] = pd.to_datetime(data['date'])
years = data['date'].dt.year.unique()
print(years)

[2020 2019 2018 2017 2016 2015]


In [4]:
all_result = []

for year in years:

    df_year = data[data['date'].dt.year == year]

    summary = []

    for index, row in df_year.iterrows():
        team1 = row['team_1']
        team2 = row['team_2']
        map_name = row['_map']
        t1_score = row['t_1']
        ct2_score = row['ct_2']
        ct1_score = row['ct_1']
        t2_score = row['t_2']

        summary.append([team1, map_name, 'T', t1_score > ct2_score])
        summary.append([team2, map_name, 'CT', ct2_score > t1_score])
        summary.append([team1, map_name, 'CT', ct1_score > t2_score])
        summary.append([team2, map_name, 'T', t2_score > ct1_score])

    results_df = pd.DataFrame(summary, columns=['team', 'map', 'side', 'win'])

    win_data = results_df.groupby(['team', 'map', 'side']).agg(
    total_matches=('win', 'size'),
    wins=('win', 'sum')
    ).reset_index()

    win_data['win_rate'] = win_data['wins'] / win_data['total_matches']

    final = win_data.pivot_table(index=['team', 'map'], columns='side', values=['total_matches', 'wins']).reset_index()
    final.columns = ['team', 'map', 'CT_total_matches', 'T_total_matches', 'CT_wins', 'T_wins']
    final['CT_win_rate'] = final['CT_wins'] / final['CT_total_matches']
    final['T_win_rate'] = final['T_wins'] / final['T_total_matches']
    final['year'] = year

    all_result.append(final)

final_results = pd.concat(all_result)
print(final_results)

            team       map  CT_total_matches  T_total_matches  CT_wins  \
0    100 Thieves     Dust2               4.0              4.0      2.0   
1    100 Thieves   Inferno               4.0              4.0      1.0   
2    100 Thieves    Mirage               2.0              2.0      1.0   
3    100 Thieves      Nuke               3.0              3.0      2.0   
4    100 Thieves     Train               1.0              1.0      0.0   
..           ...       ...               ...              ...      ...   
175  mousesports     Dust2               2.0              2.0      1.0   
176  mousesports   Inferno               2.0              2.0      1.0   
177  mousesports    Mirage               4.0              4.0      4.0   
178  mousesports  Overpass               2.0              2.0      2.0   
179  mousesports     Train               1.0              1.0      0.0   

     T_wins  CT_win_rate  T_win_rate  year  
0       2.0     0.500000    0.500000  2020  
1       2.0     0.250

In [5]:
minimum = final_results['T_total_matches'].min()
maximum = final_results['T_total_matches'].max()
print(minimum,maximum)

1.0 86.0


In [6]:
map_data = final_results.to_json(orient='records')

with open('data/map_statistics.json', 'w') as f:
    f.write(map_data)

In [6]:
df = pd.read_csv('data/picks.csv')

In [8]:
ban_columns = ['t1_removed_1', 't1_removed_2', 't1_removed_3', 't2_removed_1', 't2_removed_2', 't2_removed_3']
ban_data = []

for index, row in df.iterrows():
    team_1 = row['team_1']
    team_2 = row['team_2']
    for col in ban_columns[:3]:
        if row[col] != '0.0':
            ban_data.append({'team': team_1, 'map': row[col]})
    for col in ban_columns[3:]:
        if row[col] != '0.0':
            ban_data.append({'team': team_2, 'map': row[col]})

ban_df = pd.DataFrame(ban_data)
print(ban_df)

              team       map
0          TeamOne   Vertigo
1          TeamOne     Train
2          Recon 5      Nuke
3          Recon 5  Overpass
4          Rugratz     Dust2
...            ...       ...
75135  Tempo Storm     Train
75136  Tempo Storm   Inferno
75137         Envy  Overpass
75138         Envy    Mirage
75139         Envy     Dust2

[75140 rows x 2 columns]


In [9]:

# counting the number of times each team banned a map 
ban_counts = ban_df.groupby(['team', 'map']).size().reset_index(name='ban_count')

# find most disliked maps
def find_most_banned_maps(df):
    max_ban_count = df['ban_count'].max()
    return df[df['ban_count'] == max_ban_count]

most_disliked_maps = ban_counts.groupby('team').apply(find_most_banned_maps).reset_index(drop=True)
print(most_disliked_maps)

                team          map  ban_count
0        100 Thieves     Overpass         18
1        100pinggods         Nuke          1
2        100pinggods     Overpass          1
3        100pinggods        Train          1
4               1337  Cobblestone         17
...              ...          ...        ...
1871  x6tence Galaxy        Train         30
1872             xTc       Mirage          3
1873         zARLANS         Nuke         17
1874             zzz      Inferno          4
1875             zzz       Mirage          4

[1876 rows x 3 columns]


C:\Users\guixu\AppData\Local\Temp\ipykernel_26840\2993138514.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  most_disliked_maps = ban_counts.groupby('team').apply(find_most_banned_maps).reset_index(drop=True)


In [10]:
# save the data in json
ban_map = most_disliked_maps.to_json(orient='records')

with open('data/most_disliked_maps.json', 'w') as f:
    f.write(ban_map)

In [12]:
diffmaps = most_disliked_maps['map'].unique()
print(diffmaps)

['Overpass' 'Nuke' 'Train' 'Cobblestone' 'Vertigo' 'Cache' 'Dust2'
 'Mirage' 'Inferno']
